# 361 A2 Toby Harvie thar439 592248414
## Section 1: Report
#### Data Representation

The way I decided to represent the text was to have each word/token as an attribute, with a value defined by the number of positions of that word/token in all instances of the class in the training set. Thus, I took into consideration the frequency of the word. The reasoning behind this is that the frequency of a word could definitely correlate to a class - e.g. if a certain word is repeated a lot, it could indicate a certain class rather than if it is mentioned once or twice by chance.

A dictionary of attributes and frequency counts for each class was used for data representation as this allows O(1) lookup for frequency counts. Given the large number of lookups that occur in the Naive Bayes classification step, this is much faster than using other methods such as pandas table.

#### Investigating Preprocessing Methods and Model Improvements

A number of improvements were investigated. Those that showed improvement in the cross validation were included in the final model. I outline them all below. Most of these improvements were suggestions taken from https://people.csail.mit.edu/jrennie/papers/icml03-nb.pdf. Information was also taken from https://www.cs.cmu.edu/~tom/files/MachineLearningTomMitchell.pdf, https://www.diva-portal.org/smash/get/diva2:839705/FULLTEXT01.pdf and https://web.stanford.edu/~jurafsky/slp3/4.pdf.

##### 1. (Preprocessing) Stopwords and ngrams
For preprocessing, I removed stopwords - that is, common words such as "the" which should not have much influence on the classification but which could introduce bias. 

Another preprocessing step that I used in my final model was the use of n-grams. The idea behind this is that groups of words may infer more meaning than words on their own. For example, "machine learning" could be a more significant attribute that the attributes "machine" and "learning" appearing separately. Thus, I included all bigrams as attributes in the model. However, I thought that it would still be important to include unigrams, as otherwise single words that hold importance would lose their weight in the model - they would all become part of a potentially large number of different ngrams. See experiment 1 to see that the performance of the extended model decreases without the inclusion of bigrams: performance dropped to only 94% accuracy. The inclusion of trigrams was not found to increase performance.

##### 2. Laplace Smoothing
I implemented standard Laplace Smoothing, multiplying the frequency of each token $i$ by $\frac{N_{ci}+1}{N_c+|V|}$, where $N_{ci}$ is the number of times the token appears in the class training data, $N_c$ is the total word positions in the class training data, and $|V|$ is the size of the vocabulary. Experiment 6 shows that including this has increased performance compared to the standard model (97% compared to 90%).

##### 3. Hyperparameter in Laplace Smoothing
A common adjustment to Laplace Smoothing that I read in the literature is to introduce hyperparamter $\alpha$ to adjust the equation to $\frac{N_{ci}+\alpha}{N_c+\alpha|V|}$. Such tuning could give better likelihood estimates. I experimented with varying $\alpha$ in the extended model. Since small changes would likely overfit to the training data, I tested $\alpha$ values with step size $0.25$. I found that $\alpha=0.5$ gave the best improvement to the model of the values tested, with 98% accuracy. The next highest value was 0.75, with 97.22%, showing optimality of the chosen $\alpha$ value. Furthermore, the standard Laplace smoothing which uses $\alpha=1$ in the extended model had only 96.6%. It was important to choose this based on the extended model to account for variations caused by other extensions. See experiment 2 below. 

##### 4. Transforming by text frequency
A problem with the classification as it is could be that rare words have little impact on the most probable class. However, in reality, they could be words that appear only in a certain class and thus be a good indicator. Conversely, common words may be less likely to influence the training. Thus, I implemeneted a transformation taken directly from section 4.2 of https://people.csail.mit.edu/jrennie/papers/icml03-nb.pdf, which upweights rare words and downweights common words. However, this was shown to have incredibly poor performance, getting close to 0%. Perhaps since correlation analysis has already been imlemented, this did not the desired affect.

##### 5. (Preprocessing) Feature selection by correlation analysis
I used the chi-squared test with each feature to calculate their correlation to the class labels. Sorting by their correlation, I could remove the least correlated features from the data, which reduces dimensionality and ensures that the features being used in prediction likely have an impact on classification. I chose to include only the top 100000 attributes (there were about 160000 originally) - experiment 5 shows that this is a good hyperparameter selection (giving 98% accuracy), with cutoffs below the 100000 attribute mark having lower accuracy (<98%). Experiment 4 shows that removing correlation analysis from the extended model had a noticeable decrease in performance, decreasing accuracy by about 0.5 percentage points from the extended model's performance. Among large amounts of data this is a noticeable increase which should be implemented. 


### Evaluation Procedure
I used $k$-fold (with $k=10$) cross validation in order to evaluate the accuracy of the model using only the training data. This is a common cross validation technique which splits the data into $k$ folds. For any improvements that I considered making, I could then evaluate the model using $k$-fold evaluation before and after making the changes. See the results above and experiments below. I implemented this without any libraries in the cross_validation function in the code. In this function, the training data is randomly shuffled to ensure that the train/test split has no bias, then 10% of the training data is chosen as test data each time. The model is trained on the remaining data and then we can directly compare predictions of the synthetic data to the class labels. In this way, we have an unbiased approximation of the test data that can be used to evaluate the model.

### Final Results
The Standard Naive Bayes had an accuracy of 91.34% using cross validation and was untested on the test set.

The Extended Naive Bayes had an accuracy of 98.11% using cross validation and achieved slightly better on the test set, with 98.18%. This is a significant improvement (about 7%) over the standard model and overall has a high accuracy for the given task. Given that the cross-validation accuracy and test data accuracy are very similar (in fact, the test accuracy is slightly higher), it is likely that overfitting to the training data is not occurring.

The optimality of the final extended algorithm is shown by results dicussed above.

# Section 2: Code and Experiments



In [ ]:
import pandas as pd
import numpy as np
import os
from collections import Counter
from nltk.corpus import stopwords
from collections import defaultdict
import json
import csv
stopwords = stopwords.words('english')

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
classes = ['W', 'A', 'S', 'G']


Below is the standard Naive Bayes implementation.

In [ ]:
class standard_naive_bayes:
    # standard naive bayes implementation
    def __init__(self, train_df=pd.read_csv('train.csv'), test_df=pd.read_csv('test.csv')):
        self.train_df = train_df
        self.test_df = test_df
        self.create_vocabulary()
        self.create_freq_table()

    def get_tokens(self, desc):
        # gets tokens from a training instance
        return desc.split()

    def create_vocabulary(self):
        # creates a set of tokens from all instances of training data
        self.vocab = set()
        for index, row in self.train_df.iterrows():
            tokens = self.get_tokens(row['Description'])
            self.vocab.update(tokens)

    def create_freq_table(self):
        # creates a table of token freqencies for each class in the training data

        self.freq_table = {label : {} for label in classes}

        for index, row in self.train_df.iterrows():
            tokens = self.get_tokens(row['Description'])
            for token in tokens:
                # add or increment token count to frequency table
                self.freq_table[row['Class']][token] = self.freq_table[row['Class']].get(token,0) + 1
    
    def get_posterior(self, label, freqs, tokens):
        # gets the posterior probability P(label)P(w|label)

        tokens = Counter(tokens)
        # P(label). Prior probability
        p = np.log(len(self.train_df[self.train_df['Class']==label])/len(self.train_df))

        denom = sum(freqs.values())

        for token, count in tokens.items():
            # P(w|label). Likelihood
            if token in freqs.keys():
                p += count * np.log(freqs[token]/denom)
            else:
                p += count * np.log(1/denom)

        return p

    def classify(self, desc):
        # classifies a new instance
        probs = {}

        for label, freqs in self.freq_table.items():
            # iterate for each class label
            tokens = self.get_tokens(desc)
            probs[label] = self.get_posterior(label, freqs, tokens)
        
        # get the class corresponding to the maximum probability
        return max(probs, key=probs.get)        

    def predict(self, record_data=False):
        # computes classifications for each instance in the test set and returns the predictions

        preds = []
        for index, row in self.test_df.iterrows():
            preds.append(self.classify(row['Description']))

        if record_data:
            # write to csv
            preds_df = pd.DataFrame({'Class':preds})
            preds_df.index = np.arange(1, len(preds_df) + 1)
            preds_df.to_csv('v13.csv', index_label='Id')

        return preds

Below are methods used to cross validate

In [34]:

def get_preds(model, cross_val_train, cross_val_test):
    # standard method of getting predictions for the cross_validate method. Can adjust depending on experiment
    return model(train_df=cross_val_train, test_df=cross_val_test).predict()

def cross_validate(model, get_preds=get_preds, k=10, verbalize=True):
    # we use kfold validation as discussed in the report

    ins = len(train_df)
    accuracies = []

    for i in range(k):
        # split data
        cross_val_train = pd.concat((train_df.iloc[:int(i*ins//k)], train_df.iloc[int((i+1)*ins//k):]))
        cross_val_test = train_df.iloc[int(i*ins//k):int((i+1)*ins//k)]

        # get predictions from model based on train/test split
        preds = get_preds(model, cross_val_train, cross_val_test)

        # compute accuracy
        matches = sum(1 for i in range(len(cross_val_test)) if preds[i] == cross_val_test.iloc[i]['Class'])
        accuracy = (matches / len(cross_val_test)) * 100 
        accuracies.append( accuracy )
        if verbalize: print(f"Fold {i}: accuracy {accuracy}")

    print(f"Mean accuracy: {np.mean(accuracies)}")
    return np.mean(accuracies)

We evaluate the performance of the standard model below:

In [35]:
cross_validate(standard_naive_bayes)

Fold 0: accuracy 90.9090909090909
Fold 1: accuracy 90.68181818181819
Fold 2: accuracy 91.5909090909091
Fold 3: accuracy 92.5
Fold 4: accuracy 89.77272727272727
Fold 5: accuracy 90.22727272727272
Fold 6: accuracy 92.04545454545455
Fold 7: accuracy 91.81818181818183
Fold 8: accuracy 92.5
Fold 9: accuracy 91.36363636363637
Mean accuracy: 91.3409090909091


91.3409090909091

Below is the extended naive bayes implementation

In [36]:
class extended_naive_bayes(standard_naive_bayes):
    # extended naive bayes model
    # extensions include:
    # including bigrams and removing stopwords
    # attribute selection using chi-squared correlation analysis
    # fine tuned Laplace smoothing
    def __init__(self,train_df=pd.read_csv('train.csv'), test_df=pd.read_csv('test.csv'), record_data=False):
        super().__init__(train_df, test_df)
        self.cutoff=100000
        self.attribute_selection()
        self.record_data = record_data
        self.alpha = 0.5

    def get_tokens(self, desc):
        # changes: implementing bigrams and removing stopwords
        words = desc.split()
        words = [word for word in words if word not in stopwords]
        bigrams= [words[i-1]+'_'+words[i] for i in range(1,len(words))]
        return words + bigrams
    
    def attribute_selection(self):
        # attribute selection using chi-squared
        # implemented using information from lectures, https://www.geeksforgeeks.org/ml-chi-square-test-for-feature-selection/ and wikipedia

        # total number of token positions in each class
        class_totals = {label: sum(freqs.values()) for label, freqs in self.freq_table.items()}
        total_instances = sum(class_totals.values())

        # total token frequency across classes
        attr_totals = defaultdict(int)
        for label, freqs in self.freq_table.items():
            for token, freq in freqs.items():
                attr_totals[token] += freq

        chi_squared_scores = {}
        for attr in attr_totals:
            # for each attribute compute Chi-squared
            chi_squared = 0
            for label in self.freq_table:
                # actual number of attribute instances in class
                observed = self.freq_table[label].get(attr, 0)
                # expected number of attribute instances based on ealier
                expected = (class_totals[label] * attr_totals[attr]) / total_instances
                if expected > 0:
                    # basic chi squared formula
                    chi_squared += ((observed - expected) ** 2) / expected
            chi_squared_scores[attr] = chi_squared

        # sort attributes by score
        sorted_attrs = sorted(chi_squared_scores.items(), key=lambda x: x[1], reverse=True)
        # select top correlating attributes
        sorted_attrs = sorted_attrs[:self.cutoff]
        # get just a list of attribute names
        sorted_attrs = [x[0] for x in sorted_attrs]

        # remove uncorrelated features from frequency table
        for label in self.freq_table.keys():
            self.freq_table[label] = {attr: self.freq_table[label][attr] for attr in sorted_attrs if attr in self.freq_table[label].keys()}
    
    def get_posterior(self, label, freqs, tokens):
        # changes: Laplace smoothing. scale with parameter alpha

        tokens = Counter(tokens)
        # P(label). Prior probability
        p = np.log(len(self.train_df[self.train_df['Class']==label])/len(self.train_df))

        # denominator used in likelihood calculation
        denom = sum(freqs.values()) + self.alpha * len(self.vocab)

        # add log likelihood of each token to total probability
        for token, count in tokens.items():
            # P(w|label). Likelihood
            if token in freqs.keys():
                p += count * np.log((freqs[token] + self.alpha) / denom)
            else:
                p += count * np.log(1/denom)

        return p
                

We evaluate the performance of this model below:

In [37]:
cross_validate(extended_naive_bayes)

Fold 0: accuracy 98.4090909090909
Fold 1: accuracy 97.95454545454545
Fold 2: accuracy 97.72727272727273
Fold 3: accuracy 98.4090909090909
Fold 4: accuracy 97.27272727272728
Fold 5: accuracy 98.18181818181819
Fold 6: accuracy 98.4090909090909
Fold 7: accuracy 97.95454545454545
Fold 8: accuracy 98.4090909090909
Fold 9: accuracy 98.4090909090909
Mean accuracy: 98.11363636363636


98.11363636363636

(Experiment 1) Here we show that if we consider only unigrams on the extended model we get a lower accuracy.

In [38]:
class unigram_naive_bayes(extended_naive_bayes):
    def get_tokens(self, desc):
        # changes: removing bigrams
        words = desc.split()
        words = [word for word in words if word not in stopwords]
        #bigrams= [words[i-1]+'_'+words[i] for i in range(1,len(words))]
        return words 

cross_validate(unigram_naive_bayes)  

Fold 0: accuracy 94.77272727272728
Fold 1: accuracy 93.86363636363636
Fold 2: accuracy 94.31818181818183
Fold 3: accuracy 95.68181818181817
Fold 4: accuracy 92.27272727272727
Fold 5: accuracy 92.5
Fold 6: accuracy 93.4090909090909
Fold 7: accuracy 93.63636363636364
Fold 8: accuracy 95.0
Fold 9: accuracy 94.31818181818183
Mean accuracy: 93.97727272727273


93.97727272727273

(Experiment 2) The experiment below finds the best hyperparamter $\alpha$ for the adjusted Laplaced Smoothing. We can see that $\alpha=0.5$ produces the highest accuracy. Importantly, the common Laplace smoothing which uses $\alpha=1$ performs fairly worse (1.5 percentage points) than choosing a different value.

In [40]:
for a in [0.25*i for i in range(1,6)]:
    # iterate through alpha values

    # modified prediction function using a specified alpha value
    def get_preds_adjusted_alpha(model, cross_val_train, cross_val_test):
        nb = model(train_df = cross_val_train, test_df = cross_val_test, record_data=False)
        nb.alpha = a
        return nb.predict()

    print(f"Beginning validation for alpha={a}")
    cross_validate(extended_naive_bayes, get_preds_adjusted_alpha, k=5, verbalize=False)

Beginning validation for alpha=0.25
Mean accuracy: 94.88636363636365
Beginning validation for alpha=0.5
Mean accuracy: 98.0
Beginning validation for alpha=0.75
Mean accuracy: 97.18181818181817
Beginning validation for alpha=1.0
Mean accuracy: 96.63636363636364
Beginning validation for alpha=1.25
Mean accuracy: 96.20454545454545


(Experiment 3) The experiment below examines the affect of transforming by text frequency as discussed in the report. It implements this transformation on the already-improved model, and we see by the cross validation results that accuracy is not improved.

In [41]:
class nb_text_freq_transform(extended_naive_bayes):

    def __init__(self,train_df=pd.read_csv('train.csv'), test_df=pd.read_csv('test.csv'), record_data=False):
        # modified to create the instance table
        super().__init__(train_df, test_df)
        self.create_inst_table()

    def create_inst_table(self):
        # records number of documents in which token appears
        self.inst_table = {}
        for index, row in self.train_df.iterrows():
            tokens = self.get_tokens(row['Description'])
            for token in list(set(tokens)):
                # add or increment token instance to table of instances
                self.inst_table[token] = self.freq_table[row['Class']].get(token,0) + 1

    def get_posterior(self, label, freqs, tokens):
        # changes: transformation by text frequency

        tokens = Counter(tokens)
        # P(label). Prior probability
        p = np.log(len(self.train_df[self.train_df['Class']==label])/len(self.train_df))

        denom = sum(freqs.values()) + self.alpha * len(self.vocab)

        for token, count in tokens.items():
            # P(w|label). Likelihood
            if token in freqs.keys():
                # adjusted count function as described in the literature
                count_adjusted = count * np.log( len(self.train_df)/self.inst_table[token])
                p += count_adjusted * np.log((freqs[token] + self.alpha) / denom)
            else:
                p += count * np.log(1/denom)

        return p
    
cross_validate(nb_text_freq_transform)                

Fold 0: accuracy 0.0
Fold 1: accuracy 0.0
Fold 2: accuracy 0.0
Fold 3: accuracy 0.0
Fold 4: accuracy 0.0
Fold 5: accuracy 0.0
Fold 6: accuracy 0.0
Fold 7: accuracy 0.0
Fold 8: accuracy 0.0
Fold 9: accuracy 0.22727272727272727
Mean accuracy: 0.022727272727272728


0.022727272727272728

(Experiment 4) Here we show that the use of feature selection has a marginal influence on accuracy. Interestingly, notice that some of these scores are the same as those achieved without feature selection. This could be due 

In [42]:
class naive_bayes_no_feature_selection(extended_naive_bayes):
    
    def attribute_selection(self):
        # no attribute selection occurs
        return
    
cross_validate(naive_bayes_no_feature_selection)

Fold 0: accuracy 98.4090909090909
Fold 1: accuracy 97.72727272727273
Fold 2: accuracy 97.27272727272728
Fold 3: accuracy 98.18181818181819
Fold 4: accuracy 97.5
Fold 5: accuracy 98.18181818181819
Fold 6: accuracy 98.63636363636363
Fold 7: accuracy 97.72727272727273
Fold 8: accuracy 98.63636363636363
Fold 9: accuracy 98.4090909090909
Mean accuracy: 98.06818181818183


98.06818181818183

(Experiment 5) We search for a tuned hyperparamter for how many attributes to keep after implementing the chi squared algorithm. We see that a cutoff value over 100000 have similar accuracies, perhaps indicating that there are somewhere around 100000 important attributes to include. Ultimately, I choose a cutoff 100000 for the final model.

In [43]:
class nb_chi_experiment(extended_naive_bayes):
    # removed automatic feature selection in order to preset the cutoff value
    def __init__(self,train_df=pd.read_csv('train.csv'), test_df=pd.read_csv('test.csv'), record_data=False):
        super().__init__(train_df, test_df)
        self.record_data = record_data
        self.alpha = 0.5

#cross_validate(back_to_basics)
for a in [20000, 60000, 100000, 140000]:

    def get_preds_adjusted_chi(model, cross_val_train, cross_val_test):
        nb = model(train_df = cross_val_train, test_df = cross_val_test, record_data=False)
        nb.cutoff = a
        nb.attribute_selection()
        return nb.predict()

    print(f"Beginning validation for cutoff={a}")
    # k = 5 used to increase speed
    cross_validate(nb_chi_experiment, get_preds_adjusted_chi, k=5, verbalize=False)


Beginning validation for cutoff=20000
Mean accuracy: 96.43181818181817
Beginning validation for cutoff=60000
Mean accuracy: 97.86363636363637
Beginning validation for cutoff=100000
Mean accuracy: 98.0
Beginning validation for cutoff=140000
Mean accuracy: 98.0


(Experiment 6) Here we show that the use of Laplace smoothing improved performance compared to the standard model. With only this change, we see a significant improvement from the base model.

In [44]:
class nb_laplace_smoothing(standard_naive_bayes):
    
    def get_posterior(self, label, freqs, tokens):
        # changes: Laplace smoothing. scale with parameter alpha

        tokens = Counter(tokens)
        # P(label). Prior probability
        p = np.log(len(self.train_df[self.train_df['Class']==label])/len(self.train_df))

        # denominator used in likelihood calculation
        denom = sum(freqs.values()) + len(self.vocab)

        for token, count in tokens.items():
            # P(w|label). Likelihood
            if token in freqs.keys():
                p += count * np.log((freqs[token]) / denom)
            else:
                p += count * np.log(1/denom)

        return p

cross_validate(nb_laplace_smoothing)

Fold 0: accuracy 97.95454545454545
Fold 1: accuracy 97.27272727272728
Fold 2: accuracy 96.81818181818181
Fold 3: accuracy 97.72727272727273
Fold 4: accuracy 95.9090909090909
Fold 5: accuracy 97.95454545454545
Fold 6: accuracy 97.04545454545455
Fold 7: accuracy 96.5909090909091
Fold 8: accuracy 97.5
Fold 9: accuracy 96.13636363636363
Mean accuracy: 97.0909090909091


97.0909090909091